In [38]:
import numpy as np

import sys
from pathlib import Path

from scipy.io import savemat

# Add project paths
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import experiment management from HDMF src folder
from simulation_runner import HDMFSimulationRunner
from experiment_manager import ExperimentManager
from plotting import HDMFResultsPlotter
from utils.data_loading import load_all_sc_matrices

experiment_manager = ExperimentManager(project_root, config_path="Homeostatic_Grid", results_dir="/network/iss/cohen/data/Ivan/fastHDMF")
res = experiment_manager.load_experiment_results()
plotter = HDMFResultsPlotter(experiment_manager)
config = experiment_manager.current_config

2025-09-17 12:27:56,649 - hdmf_experiment_Homeostatic_Grid - INFO - Experiment 'Homeostatic_Grid' initialized
2025-09-17 12:27:56,651 - hdmf_experiment_Homeostatic_Grid - INFO - Config: Homeostatic grid for objective rate 3.44. Used to then calculate LR->DECAY coefficients. Now supports custom parameter generation functions.
2025-09-17 12:27:56,652 - hdmf_experiment_Homeostatic_Grid - INFO - Results will be saved to: /network/iss/cohen/data/Ivan/fastHDMF/Homeostatic_Grid
2025-09-17 12:27:56,669 - hdmf_experiment_Homeostatic_Grid - INFO - Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Homeostatic_Grid.yaml


INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Homeostatic_Grid.yaml


In [39]:
hom_grid = np.stack(res['full_results']['observables']['mean_rates'].flatten())
hom_grid = np.reshape(hom_grid, (100,110,17,90))
hom_grid = 3.44 - np.mean(hom_grid, axis=-1)
hom_grid_mean = np.mean(hom_grid, axis=2)
hom_grid_std = np.std(hom_grid, axis=2)


In [40]:
# Parameter ranges from config (fallbacks if missing)

def _range_from_config(name: str, config: dict = None) -> np.ndarray:
    # Prefer ranges computed by parent class (utils.plotting extracts grid ranges)
    
    # Check explicit Homeostatic_Grid-like config where functions are used
    grid_cfg = config.get('grid', {})
    if name in grid_cfg:
        cfg = grid_cfg[name]
        # If using function def: {fun: 'np.logspace', args: [...]}
        if isinstance(cfg, dict) and 'fun' in cfg:
            fun = cfg['fun']
            args = cfg.get('args', [])
            if fun == 'np.logspace':
                return np.logspace(*args)
            elif fun == 'np.linspace':
                return np.linspace(*args)
        
        # If using start/end/step
        if isinstance(cfg, dict) and all(k in cfg for k in ('start', 'end', 'step')):
            start, end, step = cfg['start'], cfg['end'], cfg['step']
            # inclusive-ish end
            return np.arange(start, end + step/2, step)
        # If list of values
        if isinstance(cfg, (list, tuple)):
            return np.asarray(cfg)
    
over_cfg = config.get('simulation', {}).get('over', {})
G_range = np.arange(over_cfg['range']['start'],over_cfg['range']['end'],over_cfg['range']['step'])
LR_range = _range_from_config('lrj',config=config)
DECAY_range = _range_from_config('taoj',config=config)

# Determine pure-decades ticks (first three decades) for LR and DECAY
exp_min_lr = int(np.floor(np.log10(LR_range[0])))
lr_ticks = []
lr_labels = []
for e in range(exp_min_lr, exp_min_lr + 3):
    val = 10**e
    idx = int(np.argmin(np.abs(LR_range - val)))
    lr_ticks.append(idx)
    lr_labels.append(f"{val:g}")

exp_min_dec = int(np.floor(np.log10(DECAY_range[0])))
decay_ticks = []
decay_labels = []
for e in range(exp_min_dec, exp_min_dec + 3):
    val = 10**e
    idx = int(np.argmin(np.abs(DECAY_range - val)))
    decay_ticks.append(idx)
    decay_labels.append(f"{val:g}")

nlr = len(LR_range)
ndec = len(DECAY_range)
nG = len(G_range)

if hom_grid.ndim != 3:
    raise ValueError(f"Each homeostatic grid must be 3D, got {hom_grid.shape}")
# find g axis by size match
axes_sizes = hom_grid.shape
print("Assuming G Axis is the last axis")
hom_grid_mean = np.mean(hom_grid, axis=-1)
std_hom_grid = np.std(hom_grid, axis=-1)

# Extract minimum mismatch positions along DECAY for each LR index
# In notebook they used argmin(abs(x)) over axis=0, where x is mean over G -> (LR, DECAY)
min_mm_pos = np.argmin(np.abs(hom_grid_mean), axis=1)

x_idx = np.arange(nlr)
y = min_mm_pos
if config.get('simulation', {}).get('obj_rate', 3.44) == 1.22:
    coeff = np.polyfit(LR_range[21:], y[21:], 1)
    coeff_plot = np.polyfit(x_idx[21:], y[21:], 1)
else:
    coeff = np.polyfit(LR_range, y, 1)
    coeff_plot = np.polyfit(x_idx, y, 1)

print("Fit coefficients (LR, DECAY):", coeff)
np.save(project_root / Path("data") / f"fit_res_{str(config['simulation']['obj_rate']).replace('.','-')}.npy", coeff)
savemat(project_root / Path("data") / f"fit_res_{str(config['simulation']['obj_rate']).replace('.','-')}.mat", {'fit_res': coeff})


Assuming G Axis is the last axis
Fit coefficients (LR, DECAY): [-0.07419012 65.75771659]
